# HW1

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
pos_comments = []
neg_comments = []

# скачивание отзывов с 22-х страниц
for i in range(1, 22):
    req = 'https://otziv-otziv.ru/katalog/naushniki-i-bluetooth-garnitury/xiaomi/besprovodnye-naushniki-xiaomi-airdots-pro-mi-true-wireless-earphones.html?page=' + str(i)
    response = requests.get(req)
    if (response.status_code == 200):
        soup = BeautifulSoup(response.text, 'html.parser')  # варка супа

        # поиск всех блоков с отзывами
        post = soup.findAll('div', {'class': 'container-reviews'})

        for p in post:
            # последний символ тайтла — оценка от 1 до 5
            rating = int(p.find('div', {'class': 'stars-container'}).get('title')[-1])
    
            text = p.get_text()
    
            # отзывы с оценкой 3 не включены
            if rating > 3:
                pos_comments.append({'rating': rating, 'text': text})
            elif rating < 3:
                neg_comments.append({'rating': rating, 'text': text})
    else:
        print('На странице ', i, ' ошибка.')

In [3]:
print(len(neg_comments))
print(len(pos_comments))

177
355


In [4]:
# 40 последних отзывов, на которых будет проверяться алгоритм
test_sample = []

test_size = 40
t = int(-((test_size / 2) + 1))

test_sample.extend(neg_comments[t:-1])
test_sample.extend(pos_comments[t:-1])

# удаление этих отзывов из обучающей выборки
del neg_comments[t:-1]
del pos_comments[t:-1]

In [8]:
import nltk

# токенизатор отзывов. Вход — список отзывов, выход — список слов из всех отзывов

def tokenize_comments(lst):
    all_words = []
    for comment in lst:
        tokens = nltk.word_tokenize(comment['text'])
        words = [word.lower() for word in tokens if word.isalpha()]
        all_words.extend(lemmatize(words))
    
    return all_words

In [6]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# лемматизатор. Вход — лист токенов, выход — список лемм
def lemmatize(lst):
    res = []
    for word in lst:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return res

In [9]:
tokenized_neg = tokenize_comments(neg_comments)
tokenized_pos = tokenize_comments(pos_comments)

In [10]:
# создание списков слов, которые есть только в нег. или только в поз.
def only(lst1, lst2):
    lst_only = []
    for word in lst1:
        if word not in lst2:
            lst_only.append(word)
            
    return lst_only
        
only_neg = only(tokenized_neg, tokenized_pos)
only_pos = only(tokenized_pos, tokenized_neg)

# словарь из списков
only_lists = {'pos': only_pos, 'neg': only_neg}

In [11]:
def simple_detect(only_lists, text):
    
    COUNT = {'pos': 0, 'neg': 0}
    
    tokens = nltk.word_tokenize(text)
    words = [word.lower() for word in tokens if word.isalpha()]
    lemmas = lemmatize(words)
    
    lemmas = lemmatize(tokens)
    
    for item in only_lists:
        for word in lemmas:
            if word in only_lists[item]:
                COUNT[item] += 1
                
    max_val = max(COUNT.values())
    for k, v in COUNT.items():
        if v == max_val:
            return k

При испытании на 10-ти отзывах accuracy был 100%, что кажется, скорее, случайностью, чем показателем качества. Поэтому тестовая выборка была увеличина до 40 отзывов (~10% от общего числа отзывов).

In [13]:
accuracy = 0
mistakes = 0
all_comments = 0

for comment in test_sample:
    real_value = ''
    all_comments += 1
    
    if comment['rating'] > 3:
        real_value = 'pos'
    else:
        real_value = 'neg'
    
    predict = simple_detect(only_lists, comment['text'])
    
    if predict == real_value:
        print('Успешно. Оценка: ', real_value)
        
    else:
        mistakes += 1
        print('Ошибка! Предположение: ', predict, ' Истинная оценка: ', real_value)

accuracy = (all_comments - mistakes)/ all_comments
print('\nAccuracy: ', accuracy)

Успешно. Оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Ошибка! Предположение:  pos  Истинная оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Ошибка! Предположение:  pos  Истинная оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  neg
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos
Успешно. Оценка:  pos

Accuracy:  0.95


### Предложение по исправлению:

1) Как в негативных, так и в позитивных отзывах пишут про достоинства и недостатки. Поскольку негативные отзывы по ошибке определяются как позитивные, в список слов негативных можно не включать текст который идет между словами достоинства (плюсы и т д) — недостатки (минусы и т д) (почти всегда сначала написано о плюсах).
2) Оценивать длину текста после слова "достоинства" и после слова "недостатки". В негативных вторая часть будет, предположительно, длинее.